In [1]:
from __future__ import annotations


import datetime
from typing import Any

import numpy as np
import xarray as xr
import metpy.calc as mpcalc
from metpy.units import units

import src.nzthermo as nzt  
# from src.nzthermo._core import nanshift



Pa = units.pascal
K = units.kelvin

isobaric = xr.open_dataset(
    "data/hrrr.t00z.wrfprsf00.grib2",
    engine="cfgrib",
    backend_kwargs={"filter_by_keys": {"typeOfLevel": "isobaricInhPa"}},
)


def get_data(**sel: Any):
    ds = isobaric.sel(**sel)
    T = ds["t"].to_numpy()  # (K) (Z, Y, X)
    Z, Y, X = T.shape
    N = Y * X
    T = T.reshape(Z, N).transpose()  # (N, Z)
    P = ds["isobaricInhPa"].to_numpy().astype(np.float32) * 100.0  # (Pa)
    Q = ds["q"].to_numpy()  # (kg/kg) (Z, Y, X)
    Q = Q.reshape(Z, N).transpose()  # (N, Z)
    Td = nzt.dewpoint_from_specific_humidity(P, Q)
    lat = ds["latitude"].to_numpy()
    lon = (ds["longitude"].to_numpy() + 180) % 360 - 180
    timestamp = datetime.datetime.fromisoformat(ds["time"].to_numpy().astype(str).item())
    extent = [lon.min(), lon.max(), lat.min(), lat.max()]

    return (P, T, Td), (Z, Y, X), (lat, lon, timestamp, extent)



Ignoring index file 'data/hrrr.t00z.wrfprsf00.grib2.9093e.idx' incompatible with GRIB file


In [2]:
mpcalc.most_unstable_parcel # <function metpy.calc.most_unstable_parcel(pressure, temperature, dewpoint, height=None, bottom=None, depth=None)>
(P, T, Td), (Z, Y, X), (latitude, longitude, timestamp, extent) = get_data(
    x = slice(None, None, 45), 
    y = slice(None, None, 45),
)


from src.nzthermo import equivalent_potential_temperature, parcel_profile_with_lcl




def most_unstable_parcel_index(
    pressure,
    temperature,
    dewpoint,
    /,
    depth: float = 30_000.0,
    height: float | None = None,
    bottom: float | None = None,
):
    if height is not None:
        raise NotImplementedError("height argument is not implemented")
        
    pressure = np.atleast_2d(pressure)
    p0 = (pressure[:, 0] if bottom is None else np.asarray(bottom))#.reshape(-1, 1)
    top = p0 - depth
    mask, = np.nonzero(nzt._ufunc.between_or_close(pressure, top, p0).astype(np.bool_).squeeze())
    t_layer = temperature[:, mask]
    td_layer = dewpoint[:, mask]
    p_layer = pressure[:, mask]
    theta_e = equivalent_potential_temperature(p_layer, t_layer, td_layer)
    return np.argmax(theta_e, axis=1)


np.testing.assert_array_equal(
    most_unstable_parcel_index(P, T, Td), 
    [mpcalc.most_unstable_parcel(P*Pa, T[i]*K, Td[i]*K)[-1] for i in range(Y*X)]
)




In [3]:
# idx = most_unstable_parcel(P, T, Td)

# mask = ~(idx[:, None] >= np.arange(Z)[None, :])

# # mask.argmin(1) <= np.arange(Z)

# P_new = np.full_like(mask, np.nan)




# # # np.argmin(~mask, axis=1)
# # # P_new

# # np.nonzero(mask)

# np.where(mask, P, np.nan)








In [4]:
# def nan_sort(mask, x):
#     x = np.sort(np.where(mask, x, -np.inf), axis=1)[:, ::-1]
#     x[x == -np.inf] = np.nan
#     return x




# nan_sort(
#     mask, P
# )
    
#     # nan_shift2(),
# # )

# # %timeit nanshift(np.where(mask, P, np.nan))
# # %timeit nan_shift2()

mpcalc.most_unstable_cape_cin

<function metpy.calc.most_unstable_cape_cin(pressure, temperature, dewpoint, **kwargs)>

In [9]:
(P, T, Td), (Z, Y, X), (latitude, longitude, timestamp, extent) = get_data(
    x = slice(None, None, 45), 
    y = slice(None, None, 45),
)




idx = most_unstable_parcel_index(P, T, Td)

mask = (idx[:, None] <= np.arange(Z)[None, :])



sort = np.argsort(np.where(mask, P, -np.inf))


P_layer, T_layer, Td_layer = np.take_along_axis(
    np.asarray([
        np.where(mask, P, np.nan), 
        np.where(mask, T, np.nan),  
        np.where(mask, Td, np.nan),
    ]),
    sort[np.newaxis, :, :],
    axis=-1,
)[:, :, ::-1]


# environment pressure, environment temperature, environment dewpoint, parcel temperature profile
ep, et, etd, ptp = parcel_profile_with_lcl(P_layer, T_layer, Td_layer)
from numpy.testing import assert_allclose
# print(ep, et, etd, ptp, sep='\n')
for i in range(0, Y*X, 5): 
    parcel_idx = idx[i]
    ep_, et_, etd_, ptp_ = [x.m for x in mpcalc.parcel_profile_with_lcl(
        P[parcel_idx:] * Pa,
        T[i, parcel_idx:] * K,
        Td[i, parcel_idx:] * K,
    )]
    # nan_tail = [np.nan]*(Z+1 - len(ep_))
    nan_tail = [np.nan] * (Z+1 - len(ep_))

    ep_, et_, etd_, ptp_ = (
        np.array([*x] + nan_tail) for x in (ep_, et_, etd_, ptp_)
    )

    etd_[0] = Td[i, parcel_idx]
    ptp_[0] = T[i, parcel_idx]
    ep_[0] = P[parcel_idx]
    et_[0] = T[i, parcel_idx]

    # assert_allclose(ep[i], ep_, rtol=1e-3)

    # print(et[i], et_, sep='\n')
    assert_allclose(et[i], et_, rtol=1e-3)

    # print(etd[i], etd_, sep='\n')
    assert_allclose(etd[i], etd_, rtol=1e-3)

    # print(ptp[i], ptp_, sep="\n")
    assert_allclose(ptp[i], ptp_, rtol=1e-2)

    # break
    # assert_allclose(etd[i], etd_, rtol=1e-1)
    assert_allclose(ptp[i], ptp_, rtol=1e-2)
    # break


/tmp/ipykernel_93894/615816705.py:35: UserWarning: Interpolation point out of data bounds encountered
  ep_, et_, etd_, ptp_ = [x.m for x in mpcalc.parcel_profile_with_lcl(


AssertionError: 
Not equal to tolerance rtol=0.001, atol=0

Mismatched elements: 1 / 41 (2.44%)
Max absolute difference: 0.37127686
Max relative difference: 0.00127804
 x: array([290.5055 , 290.87677, 284.92957, 281.49792, 280.63437, 280.51614,
       280.40613, 280.20428, 278.98447, 276.37292, 274.80176, 273.24625,
       271.04562, 268.35522, 265.32175, 262.16504, 258.94226, 256.2096 ,...
 y: array([290.505493, 290.505493, 284.929565, 281.497925, 280.634369,
       280.516144, 280.406128, 280.204285, 278.984467, 276.372925,
       274.801758, 273.246246, 271.045624, 268.355225, 265.321747,...